In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [2]:
! pip install nlp shap

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 38.0 MB/s eta 0:00:00


In [3]:
import nlp
import numpy as np
import scipy as sp
import torch
from torch import nn
import transformers
from transformers import BertModel, BertTokenizer
import shap
from torch.utils.data import Dataset, DataLoader

/opt/conda/lib/python3.10/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.24.3
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


In [4]:
imdb_train = nlp.load_dataset("imdb")["train"]

Downloading:   0%|          | 0.00/4.56k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/2.07k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/84.1M [00:00<?, ?B/s]

0 examples [00:00, ? examples/s]

0 examples [00:00, ? examples/s]

0 examples [00:00, ? examples/s]

Dataset imdb downloaded and prepared to /root/.cache/huggingface/datasets/imdb/plain_text/1.0.0/76cdbd7249ea3548c928bbf304258dab44d09cd3638d9da8d42480d1d1be3743. Subsequent calls will reuse this data.


In [5]:
set(imdb_train['label'])

{0, 1}

In [6]:
df2 = pd.DataFrame(imdb_train)
df_p1 = df2[df2.label==1][:500]
df_p2 = df2[df2.label==0][:500]
df = pd.concat((df_p1,df_p2),axis=0)
del df2, df_p1, df_p2

In [7]:
df.head()

,label,text
0,1,Bromwell High is a cartoon comedy. It ran at t...
1,1,Homelessness (or Houselessness as George Carli...
2,1,Brilliant over-acting by Lesley Ann Warren. Be...
3,1,This is easily the most underrated film inn th...
4,1,This is not the typical Mel Brooks film. It wa...


In [8]:
df.label.value_counts()

label
1    500
0    500
Name: count, dtype: int64

In [9]:
MAX_LEN = 200
TRAIN_BATCH_SIZE = 8
VALID_BATCH_SIZE = 8
EPOCHS = 2
LEARNING_RATE = 1e-05
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

tokenizer_config.json:   0%|          | 0.00/28.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

In [10]:
class CustomDataset(Dataset):
    def __init__(self, dataframe, tokenizer, max_len):
        self.tokenizer = tokenizer
        #self.data = dataframe
        self.title = dataframe['text']
        self.targets = dataframe['label']
        self.max_len = max_len

    def __len__(self):
        return len(self.title)

    def __getitem__(self, index):
        title = str(self.title[index])
        title = " ".join(title.split())

        inputs = self.tokenizer.encode_plus(
            title,
            None,
            add_special_tokens=True,
            max_length=self.max_len,
            padding='max_length',
            return_token_type_ids=True,
            truncation=True
        )
        ids = inputs['input_ids']
        mask = inputs['attention_mask']
        token_type_ids = inputs["token_type_ids"]


        return {
            'ids': torch.tensor(ids, dtype=torch.long),
            'mask': torch.tensor(mask, dtype=torch.long),
            'token_type_ids': torch.tensor(token_type_ids, dtype=torch.long),
            'targets': torch.tensor(self.targets[index], dtype=torch.float)
        }

In [11]:
train_size = 0.8
train_dataset = df.sample(frac=train_size,random_state=200)
train_dataset = train_dataset.iloc[:(len(train_dataset)-(len(train_dataset)%8))]
valid_dataset = df.drop(train_dataset.index).reset_index(drop=True)
valid_dataset = valid_dataset.iloc[:(len(valid_dataset)-(len(valid_dataset)%8))]
train_dataset = train_dataset.reset_index(drop=True)

In [12]:
print("FULL Dataset: {}".format(df.shape))
print("TRAIN Dataset: {}".format(train_dataset.shape))
print("TEST Dataset: {}".format(valid_dataset.shape))

FULL Dataset: (1000, 2)
TRAIN Dataset: (800, 2)
TEST Dataset: (200, 2)


In [13]:
training_set = CustomDataset(train_dataset, tokenizer, MAX_LEN)
validation_set = CustomDataset(valid_dataset, tokenizer, MAX_LEN)

In [14]:
training_set[0]

{'ids': tensor([  101, 18296,  2545,  1012,  1012,  1012,  1012,  1012,  2092,  2065,
          2017,  2066, 22812,  2049,  5791,  2028,  2000,  2156,  1010,  4728,
          2123,  1005,  1056,  2130,  8572,  1012,  2009,  2003,  3294,  5793,
          2008,  2023,  2143,  2038,  2019,  5186,  2659,  5166,  1010,  2005,
          6013,  2009,  3504,  2004,  2065,  1996,  2972,  2143,  2038,  2042,
          2915,  1999,  1037,  9746,  4873,  1010,  1998,  2006,  3365,  6642,
          2017,  2097,  2156,  1996,  3505,  8797,  5192,  1998,  1996,  4950,
         16042,  5192,  1010,  3404,  2033,  2017,  2180,  2102,  2342,  2000,
          2298,  2005,  2068,  1012,  2036,  3046,  2000,  8568,  1996, 18178,
          2229,  2100,  5889,  1010,  2065,  2008,  2015,  2054,  2017,  2655,
          2068,   999,   999,  1996,  3937, 12685,  2003,  1037,  2261,  2111,
          5630,  2000,  2175,  2006,  1037,  2208,  2265,  2073,  2027,  2031,
          2000,  5788,  1037,  2305,  1999,  

In [15]:
device = torch.device('cuda')
train_params = {'batch_size': TRAIN_BATCH_SIZE,
                'shuffle': True,
                'num_workers': 0
                }

test_params = {'batch_size': VALID_BATCH_SIZE,
                'shuffle': False,
                'num_workers': 0
                }

training_loader = DataLoader(training_set, **train_params)
validation_loader = DataLoader(validation_set, **test_params)

In [16]:
next(iter(training_loader))

{'ids': tensor([[ 101, 2054, 2064,  ...,    0,    0,    0],
         [ 101, 1045, 2018,  ..., 2043, 1996,  102],
         [ 101, 1996, 1000,  ..., 3517, 2009,  102],
         ...,
         [ 101, 2026, 2564,  ..., 2017, 2074,  102],
         [ 101, 5151, 2143,  ...,    0,    0,    0],
         [ 101, 1038, 3185,  ...,    0,    0,    0]]),
 'mask': tensor([[1, 1, 1,  ..., 0, 0, 0],
         [1, 1, 1,  ..., 1, 1, 1],
         [1, 1, 1,  ..., 1, 1, 1],
         ...,
         [1, 1, 1,  ..., 1, 1, 1],
         [1, 1, 1,  ..., 0, 0, 0],
         [1, 1, 1,  ..., 0, 0, 0]]),
 'token_type_ids': tensor([[0, 0, 0,  ..., 0, 0, 0],
         [0, 0, 0,  ..., 0, 0, 0],
         [0, 0, 0,  ..., 0, 0, 0],
         ...,
         [0, 0, 0,  ..., 0, 0, 0],
         [0, 0, 0,  ..., 0, 0, 0],
         [0, 0, 0,  ..., 0, 0, 0]]),
 'targets': tensor([1., 1., 0., 0., 1., 0., 1., 0.])}

In [17]:
device = torch.device('cuda')
class BERTClass(torch.nn.Module):
    def __init__(self):
        super(BERTClass, self).__init__()
        self.l1 = BertModel.from_pretrained('bert-base-uncased', return_dict=True)
        self.l2 = torch.nn.Dropout(0.3)
        self.l3 = torch.nn.Linear(768, 1)
    
    def forward(self, ids, mask, token_type_ids):
        output_1= self.l1(ids, attention_mask = mask, token_type_ids = token_type_ids)
        output_2 = self.l2(output_1[0][:,0,:])
        output = self.l3(output_2)
        return output

model = BERTClass()
model.to(device)

model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

BERTClass(
  (l1): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-11): 12 x BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=Tr

In [18]:
def loss_fn(outputs, targets):
    targets = targets.type(torch.LongTensor).to(device)
    criterion = nn.BCEWithLogitsLoss().cuda()
    return criterion(outputs.reshape([8,1]), targets.reshape([8,1]).float())

optimizer = torch.optim.Adam(params =  model.parameters(), lr=LEARNING_RATE)

In [19]:
def train_model(start_epochs,  n_epochs, valid_loss_min_input, 
                training_loader, validation_loader, model, 
                optimizer):
    # initialize tracker for minimum validation loss
    valid_loss_min = valid_loss_min_input
    val_targets = []

    for epoch in range(start_epochs, n_epochs+1):
        train_loss = 0
        valid_loss = 0

        model.train()
        print('############# Epoch {}: Training Start   #############'.format(epoch))
        for batch_idx, data in enumerate(training_loader):
            #print('yyy epoch', batch_idx)
            ids = data['ids'].to(device, dtype = torch.long)
            mask = data['mask'].to(device, dtype = torch.long)
            token_type_ids = data['token_type_ids'].to(device, dtype = torch.long)
            targets = data['targets'].to(device, dtype = torch.float)

            outputs = model(ids, mask, token_type_ids)

            optimizer.zero_grad()
            #print(outputs, targets)
            loss = loss_fn(outputs, targets)
            #if batch_idx%5000==0:
             #   print(f'Epoch: {epoch}, Training Loss:  {loss.item()}')

            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
            #print('before loss data in training', loss.item(), train_loss)
            train_loss = train_loss + ((1 / (batch_idx + 1)) * (loss.item() - train_loss))
            #print('after loss data in training', loss.item(), train_loss)

        print('############# Epoch {}: Training End     #############'.format(epoch))

        print('############# Epoch {}: Validation Start   #############'.format(epoch))
        ######################    
        # validate the model #
        ######################

        model.eval()

        with torch.no_grad():
          for batch_idx, data in enumerate(validation_loader, 0):
                ids = data['ids'].to(device, dtype = torch.long)
                mask = data['mask'].to(device, dtype = torch.long)
                token_type_ids = data['token_type_ids'].to(device, dtype = torch.long)
                targets = data['targets'].to(device, dtype = torch.float)
                outputs = model(ids, mask, token_type_ids)

                loss = loss_fn(outputs, targets)
                valid_loss = valid_loss + ((1 / (batch_idx + 1)) * (loss.item() - valid_loss))
                #val_targets.extend(targets.cpu().detach().numpy().tolist())
                #val_outputs.extend(torch.sigmoid(outputs).cpu().detach().numpy().tolist())

          print('############# Epoch {}: Validation End     #############'.format(epoch))
          # calculate average losses
          #print('before cal avg train loss', train_loss)
          train_loss = train_loss/len(training_loader)
          valid_loss = valid_loss/len(validation_loader)
          # print training/validation statistics 
          print('Epoch: {} \tAvgerage Training Loss: {:.6f} \tAverage Validation Loss: {:.6f}'.format(
                epoch, 
                train_loss,
                valid_loss
                ))

        print('############# Epoch {}  Done   #############\n'.format(epoch))
    return model

In [20]:
trained_model = train_model(1, 2, np.Inf, training_loader, validation_loader, model, optimizer)

############# Epoch 1: Training Start   #############
############# Epoch 1: Training End     #############
############# Epoch 1: Validation Start   #############
############# Epoch 1: Validation End     #############
Epoch: 1 	Avgerage Training Loss: 0.006366 	Average Validation Loss: 0.016987
############# Epoch 1  Done   #############

############# Epoch 2: Training Start   #############
############# Epoch 2: Training End     #############
############# Epoch 2: Validation Start   #############
############# Epoch 2: Validation End     #############
Epoch: 2 	Avgerage Training Loss: 0.003511 	Average Validation Loss: 0.012627
############# Epoch 2  Done   #############



In [21]:
def f(word_list):
    all_output = []
    for word in word_list:
        word = tokenizer.encode_plus(word, padding='max_length',truncation=True, max_length=512)
        ids = torch.tensor(word['input_ids']).flatten().unsqueeze(0).to(device, dtype = torch.long)
        attention_mask = torch.tensor(word['attention_mask']).flatten().unsqueeze(0).to(device, dtype = torch.long)
        token_type_ids = torch.tensor(word['token_type_ids']).flatten().unsqueeze(0).to(device, dtype = torch.long)
        #print(ids,attention_mask,token_type_ids)
        outputs = trained_model(ids,attention_mask,token_type_ids).squeeze().detach().cpu().numpy()
        #print(outputs)
        all_output.append(outputs)
    all_output = (all_output)
    #print(all_output)
    return all_output
#     scores = (np.exp(all_output).T / np.exp(all_output).sum(-1)).T
#     print(scores)
#     val = sp.special.logit(scores)
#     val = sp.special.logit(all_output)
    #print(val)
#     return val


In [22]:
df['text'][:3]

0    Bromwell High is a cartoon comedy. It ran at t...
1    Homelessness (or Houselessness as George Carli...
2    Brilliant over-acting by Lesley Ann Warren. Be...
Name: text, dtype: object

In [23]:
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
masker = shap.maskers.Text(tokenizer)
explainer = shap.Explainer(f, masker)

In [24]:
shap_values = explainer(df['text'][:3])
#shap_values

  0%|          | 0/498 [00:00<?, ?it/s]

PartitionExplainer explainer:  33%|███▎      | 1/3 [00:00<?, ?it/s]Token indices sequence length is longer than the specified maximum sequence length for this model (559 > 512). Running this sequence through the model will result in indexing errors


  0%|          | 0/498 [00:00<?, ?it/s]

PartitionExplainer explainer: 100%|██████████| 3/3 [00:36<00:00, 10.20s/it]

  0%|          | 0/498 [00:00<?, ?it/s]

PartitionExplainer explainer: 4it [00:50, 16.80s/it]


In [25]:
shap.plots.text(shap_values)

In [26]:
import pickle 
pickle.dump(shap_values,open('df.pkl','wb'))

In [27]:
import pickle
shap_values_3 =pickle.load(open('df.pkl','rb'))

In [28]:
shap.plots.text(shap_values_3)